In [1]:
!pip install selenium pytest webdriver-manager

In [5]:
import unittest
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

# Define the Page Object Model (POM) classes

class GoogleSearchPage:
    def __init__(self, driver):
        self.driver = driver
        self.search_box = (By.NAME, "q")  # Locator for the search box

    def open(self):
        self.driver.get("https://www.google.com")

    def enter_search_query(self, query):
        self.driver.find_element(*self.search_box).send_keys(query)

    def submit_search(self):
        self.driver.find_element(*self.search_box).send_keys(Keys.RETURN)


class SearchResultsPage:
    def __init__(self, driver):
        self.driver = driver
        self.result_stats = (By.ID, "result-stats")  # Locator for result stats

    def get_result_stats(self):
        try:
            # Wait for the element to be present
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located(self.result_stats)
            )
            return self.driver.find_element(*self.result_stats).text
        except Exception as e:
            print(f"Error: {e}")
            return "No results found"


# Define the Test Case using unittest

class GoogleSearchTest(unittest.TestCase):
    def setUp(self):
        # Use WebDriver Manager to automatically manage the ChromeDriver
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        self.driver.implicitly_wait(10)  # Set implicit wait

    def test_search_selenium(self):
        # Create page objects
        google_search_page = GoogleSearchPage(self.driver)
        search_results_page = SearchResultsPage(self.driver)

        # Open Google and search for "Selenium"
        google_search_page.open()
        google_search_page.enter_search_query("Selenium")
        google_search_page.submit_search()

        # Wait for the results page to load
        time.sleep(10)

        # Debugging: Print the page source or take a screenshot
        print(self.driver.page_source)  # Print the page source
        self.driver.save_screenshot("search_results.png")  # Take a screenshot

        # Verify results
        result_stats = search_results_page.get_result_stats()
        self.assertIn("results", result_stats)  # Assert that results are displayed

    def tearDown(self):
        self.driver.quit()  # Close the browser


# Run the Test Case in Jupyter Notebook

if __name__ == "__main__":
    # Create a test suite
    suite = unittest.TestLoader().loadTestsFromTestCase(GoogleSearchTest)

    # Run the tests using unittest
    unittest.TextTestRunner().run(suite)

<html><head><meta http-equiv="origin-trial" content="A/kargTFyk8MR5ueravczef/wIlTkbVk1qXQesp39nV+xNECPdLBVeYffxrM8TmZT6RArWGQVCJ0LRivD7glcAUAAACQeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZzIiLCJleHBpcnkiOjE3NDIzNDIzOTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9"><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://www.google.com/search?q=Selenium&amp;sca_esv=6c98adb7f3c8c910&amp;source=hp&amp;ei=AY6zZ7f_Hd-WseMPmKqoSA&amp;iflsig=ACkRmUkAAAAAZ7OcEe_QjvP7TtBlSVfBVAKXuy1K8Js9&amp;ved=0ahUKEwj3g9nMuMuLAxVfS2wGHRgVCgkQ4dUDCBA&amp;uact=5&amp;oq=Selenium&amp;gs_lp=Egdnd3Mtd2l6IghTZWxlbml1bUg9UABYFHAAeACQAQCYAQCgAQCqAQC4AQPIAQD4AQGYAgCgAgCYAwCSBwCgBwA&amp;sclient=gws-wiz&amp;sei=A46zZ7fkMpqhseMPr4zg0Aw</title></head>
<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behav

.
----------------------------------------------------------------------
Ran 1 test in 28.923s

OK


In [3]:
import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

# Define the Page Object Model (POM) classes

class GoogleSearchPage:
    def __init__(self, driver):
        self.driver = driver
        self.search_box = (By.NAME, "q")  # Locator for the search box

    def open(self):
        self.driver.get("https://www.google.com")

    def enter_search_query(self, query):
        self.driver.find_element(*self.search_box).send_keys(query)

    def submit_search(self):
        self.driver.find_element(*self.search_box).send_keys(Keys.RETURN)


class SearchResultsPage:
    def __init__(self, driver):
        self.driver = driver
        self.result_stats = (By.ID, "result-stats")  # Locator for result stats

    def get_result_stats(self):
        return self.driver.find_element(*self.result_stats).text


# Define the Test Case using pytest

@pytest.fixture(scope="function")
def driver():
    # Use WebDriver Manager to automatically manage the ChromeDriver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.implicitly_wait(10)  # Set implicit wait
    yield driver
    driver.quit()  # Close the browser after the test


@pytest.mark.parametrize("query", ["Selenium", "Python", "Automation"])
def test_search(driver, query):
    # Create page objects
    google_search_page = GoogleSearchPage(driver)
    search_results_page = SearchResultsPage(driver)

    # Open Google and search for the query
    google_search_page.open()
    google_search_page.enter_search_query(query)
    google_search_page.submit_search()

    # Verify results
    result_stats = search_results_page.get_result_stats()
    assert "results" in result_stats  # Assert that results are displayed